In [1]:
import os
import glob
import re
import numpy as np
import numpy.fft as fft
import mne
import matplotlib.pyplot as plt
from scipy import stats, signal
from numpy import save, load
import pandas as pd
import sklearn
from sklearn.svm import SVC

### read dataset X and Y

In [77]:
subject_id=1
base_path = "features/"
# edf_file_names = sorted(glob.glob(os.path.join(base_path, "data_chb01/*.npy".format(subject_id))))
# files=len(edf_file_names)
X=load('features/data_chb01/features_{}_00.npy'.format(subject_id))
y=load('features/data_chb01/targets_{}_00.npy'.format(subject_id))
for subject_id in range(1,5):
    edf_file_names = sorted(glob.glob(os.path.join(base_path, "data_chb{:02d}/*.npy".format(subject_id))))
    files=len(edf_file_names)
    print(files)
    print(X.shape)
    start=0
    if subject_id==1:
        start=1
    for fileno in range(start, files//2):
        print(fileno)
        X=np.concatenate((X, load('features/data_chb{:02d}/features_{}_{:02d}.npy'.format(subject_id, subject_id, fileno))))
        y=np.concatenate((y, load('features/data_chb{:02d}/targets_{}_{:02d}.npy'.format(subject_id,subject_id, fileno))))


84
(408, 208)
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
72
(14584, 208)
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
76
(27194, 208)
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
24
(40766, 208)
0
1
2
3
4
5
6
7
8
9
10
11


In [78]:
X.shape, y.shape

((55977, 208), (55977,))

In [79]:
df=pd.DataFrame(data=X)

In [80]:
df['target']=y

In [81]:
df.shape

(55977, 209)

In [82]:
df=df.dropna()
df.shape

(55977, 209)

In [16]:
df[(df['target']==3) | (df['target']==1)]

,0,1,2,3,4,5,6,7,8,9,...,199,200,201,202,203,204,205,206,207,target
767,0.504019,0.161749,0.036241,0.087788,0.117071,0.010672,0.075750,0.006710,35.378636,0.340825,...,0.282103,0.070293,0.147826,0.065048,0.009173,0.059392,0.005359,22.022274,1.0,3
768,0.397320,0.131061,0.041063,0.131679,0.127953,0.018591,0.123891,0.028443,41.653349,0.137482,...,0.261818,0.062877,0.153241,0.082529,0.013847,0.073221,0.010040,24.051126,1.0,3
769,0.032970,0.067123,0.011638,0.045629,0.222607,0.082048,0.388199,0.149785,359.949214,0.022584,...,0.168848,0.029229,0.121630,0.179059,0.042258,0.250080,0.064458,80.585204,1.0,3
770,0.013406,0.067360,0.010145,0.044625,0.228867,0.084977,0.397326,0.153293,342.790860,0.017106,...,0.153139,0.030927,0.122526,0.180385,0.046731,0.282134,0.073164,88.751985,1.0,3
771,0.131704,0.184924,0.037553,0.238695,0.201855,0.023583,0.158190,0.023495,23.179371,0.198627,...,0.184617,0.045994,0.168155,0.154820,0.037461,0.221824,0.067128,31.520675,1.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9151,0.024826,0.082295,0.049787,0.065915,0.295219,0.086339,0.333806,0.061814,56.292130,0.102190,...,0.048700,0.042667,0.101418,0.115535,0.100833,0.517067,0.014999,97.952720,1.0,3
9152,0.014045,0.083170,0.027623,0.031817,0.319929,0.085996,0.365671,0.071750,75.450678,0.065839,...,0.056387,0.047088,0.089704,0.125709,0.098840,0.507692,0.015631,98.250833,1.0,3
9153,0.022369,0.120699,0.032243,0.047050,0.325650,0.063891,0.319887,0.068213,45.941663,0.062855,...,0.053789,0.052068,0.111783,0.115940,0.098990,0.508954,0.013975,97.544856,1.0,3
9154,0.034451,0.159178,0.060282,0.076829,0.325747,0.056455,0.239840,0.047217,27.680383,0.085372,...,0.045370,0.047184,0.114786,0.107574,0.100771,0.512889,0.013147,94.074203,1.0,3


### train test split

In [83]:
df_interictal=df[df['target']==0]
# print(df_interictal.shape)
# df_interictal=df_interictal.sample(frac=0.7)
df_preictal=df[df['target']==2]
print(df_interictal.shape, df_preictal.shape)

(48976, 209) (2553, 209)


In [84]:
2553/(2553+48976)

0.049544916454811856

In [34]:
df_interictal['target']

0        0
1        0
2        0
3        0
4        0
        ..
14579    0
14580    0
14581    0
14582    0
14583    0
Name: target, Length: 12188, dtype: int64

In [85]:
X_interictal=np.array(df_interictal[df.columns[:-1]]).astype('float32')
y_interictal=np.array(df_interictal['target']).astype('float32')
X_preictal  =np.array(df_preictal[df.columns[:-1]]).astype('float32')
y_preictal  =np.array(df_preictal['target']).astype('float32')

In [86]:
print(X_interictal.shape)
print(y_interictal.shape)
print(X_preictal.shape)
print(y_preictal.shape)

(48976, 208)
(48976,)
(2553, 208)
(2553,)


In [89]:
130304+6870

137174

In [90]:
df_preictal[:-1]

,0,1,2,3,4,5,6,7,8,9,...,199,200,201,202,203,204,205,206,207,target
885,0.313209,0.105117,0.177755,0.170171,0.130082,0.011442,0.085478,0.006744,31.195422,0.185326,...,0.135054,0.295482,0.275806,0.049864,0.007605,0.049843,0.004499,24.849719,1.0,2
886,0.361305,0.104534,0.148806,0.178307,0.115854,0.009840,0.075175,0.006179,35.477830,0.254696,...,0.131401,0.274972,0.285203,0.052202,0.006579,0.043109,0.003889,27.723220,1.0,2
887,0.350512,0.105335,0.136584,0.175367,0.127451,0.011300,0.086123,0.007327,30.993516,0.258523,...,0.137225,0.273114,0.272945,0.057310,0.007824,0.049630,0.004442,23.797804,1.0,2
888,0.322681,0.101829,0.152082,0.189847,0.130983,0.011281,0.084018,0.007280,30.840686,0.213205,...,0.131797,0.291868,0.275333,0.058885,0.008209,0.052058,0.004627,22.406452,1.0,2
889,0.301142,0.102314,0.173153,0.207157,0.121025,0.010474,0.078348,0.006388,33.194468,0.230240,...,0.131175,0.261674,0.314244,0.050905,0.006702,0.046090,0.004024,26.524375,1.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149676,0.060026,0.192568,0.131846,0.142469,0.265760,0.037718,0.138473,0.031140,378.386978,0.021143,...,0.153421,0.005989,0.014741,0.292584,0.062299,0.333739,0.126612,1734.162379,9.0,2
149677,0.011526,0.083213,0.013058,0.018777,0.219783,0.073176,0.397635,0.182831,3452.284012,0.004761,...,0.137901,0.015380,0.035846,0.280160,0.060160,0.309436,0.136690,1060.737877,9.0,2
149678,0.009555,0.079954,0.011768,0.014612,0.208401,0.078375,0.410033,0.187302,4265.836601,0.006300,...,0.171916,0.029890,0.068148,0.327419,0.048095,0.245393,0.073114,764.756033,9.0,2
149679,0.016634,0.083771,0.033295,0.038685,0.199133,0.090117,0.399899,0.138465,902.970531,0.015999,...,0.172627,0.029499,0.076296,0.306140,0.056033,0.266259,0.068919,560.460805,9.0,2


In [87]:
from sklearn.model_selection import train_test_split


In [88]:
X_interictal_train, X_interictal_test, y_interictal_train, y_interictal_test =train_test_split(X_interictal,y_interictal,test_size=0.1, random_state=42)
X_preictal_train, X_preictal_test, y_preictal_train, y_preictal_test=train_test_split(X_preictal, y_preictal,test_size=0.1, random_state=42)

In [89]:
X_train = np.concatenate((X_interictal_train, X_preictal_train))
X_test = np.concatenate((X_interictal_test, X_preictal_test))
y_train = np.concatenate((y_interictal_train, y_preictal_train))
y_test = np.concatenate((y_interictal_test, y_preictal_test))

In [90]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(46375, 208)
(5154, 208)
(46375,)
(5154,)


### Normalization

In [16]:
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# scaler.fit(X_train)
# scaler.transform(X_train)
# scaler.fit(X_test)
# scaler.transform(X_test)

array([[5.06899767e-02, 2.06566319e-01, 2.44740620e-02, ...,
        3.16168010e-01, 8.29013061e-07, 1.00000000e+00],
       [2.38289069e-02, 2.64799237e-01, 1.16495766e-01, ...,
        1.51608899e-01, 2.98875057e-06, 0.00000000e+00],
       [1.77743211e-02, 1.85866445e-01, 1.43353671e-01, ...,
        1.70185298e-01, 9.69898610e-06, 6.66666627e-01],
       ...,
       [2.15109922e-02, 9.67502445e-02, 1.10558691e-02, ...,
        4.12113339e-01, 1.13175914e-03, 1.00000000e+00],
       [6.81196852e-03, 6.83366656e-02, 3.92035069e-03, ...,
        4.36358184e-01, 4.70467021e-05, 6.66666627e-01],
       [4.16661575e-02, 2.63042718e-01, 1.29583791e-01, ...,
        1.59748152e-01, 5.48839807e-06, 3.33333343e-01]], dtype=float32)

### training the model

In [37]:
from datetime import datetime
before = datetime.now()

before_time =before.strftime("%H:%M:%S")
print(before_time)

14:27:52


In [67]:
weight_dict={0:0.1, 2:1.0}

In [68]:
clf = SVC(class_weight=weight_dict)
clf.fit(X_train, y_train)
trainAcc = clf.score(X_train, y_train)
testAcc = clf.score(X_test, y_test)
print("**SVM Results:**")
print("Training Accuracy: %d"%(trainAcc*100)+"%")
print("Testing Accuracy: %d"%(testAcc *100)+"%")

**SVM Results:**
Training Accuracy: 68%
Testing Accuracy: 70%


In [69]:
after = datetime.now()

after_time =after.strftime("%H:%M:%S")
print(after_time)

15:45:47


In [40]:
import joblib

    # estimator = joblib.load("/my_models/%s.pkl"%dataset_name)
    # print "using trained model"

print ("saving new model")
joblib.dump(clf,"my_models/SVM_blanced_chb04.pkl")

saving new model


['my_models/SVM_blanced_chb04.pkl']

In [92]:
loaded_clf = joblib.load("my_models/SVM_blanced_chb04.pkl")
y_pred=loaded_clf.predict(X_test)

In [73]:
loaded_clf.get_params(deep=True)

{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': 'balanced',
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [93]:
tn, fp, fn, tp = sklearn.metrics.confusion_matrix(y_test, y_pred).ravel()
print("TN:{}, FP:{}, FN:{}, TP:{}".format(tn, fp, fn, tp))

TN:4844, FP:54, FN:240, TP:16


### grid search

In [101]:
from numpy import mean
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold

In [102]:
balance = [{0:10,2:1}, {0:1,2:1}, {0:1,2:10}, {0:1,2:50}, {0:1,2:100}]
param_grid = dict(class_weight=balance)

In [104]:
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=1, random_state=1)
grid = GridSearchCV(estimator=clf, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')


In [ ]:
grid_result = grid.fit(X_train, y_train)

In [ ]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))